In [39]:
import pandas as pd
import numpy as np
import json
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score



In [40]:
# Load the JSON file
with open('den.json', 'r') as f:
    json_data = json.load(f)
json_data

{'session_name': 'test',
 'session_description': 'test',
 'design_state_data': {'session_info': {'project_id': '1',
   'experiment_id': 'kkkk-11',
   'dataset': 'iris_modified.csv',
   'session_name': 'test',
   'session_description': 'test'},
  'target': {'prediction_type': 'Regression',
   'target': 'petal_width',
   'type': 'regression',
   'partitioning': True},
  'train': {'policy': 'Split the dataset',
   'time_variable': 'sepal_length',
   'sampling_method': 'No sampling(whole data)',
   'split': 'Randomly',
   'k_fold': False,
   'train_ratio': 0,
   'random_seed': 0},
  'metrics': {'optomize_model_hyperparameters_for': 'AUC',
   'optimize_threshold_for': 'F1 Score',
   'compute_lift_at': 0,
   'cost_matrix_gain_for_true_prediction_true_result': 1,
   'cost_matrix_gain_for_true_prediction_false_result': 0,
   'cost_matrix_gain_for_false_prediction_true_result': 0,
   'cost_matrix_gain_for_false_prediction_false_result': 0},
  'feature_handling': {'sepal_length': {'feature_name'

In [41]:
import pandas as pd

df1 = pd.read_json('den.json')

#Finding the missing values
missing_values = df1.isna().sum().sum()

# print the number of missing values
print(f'Number of missing values: {missing_values}')
df1

Number of missing values: 0


session_name session_description  \
algorithms                      test                test   
feature_generation              test                test   
feature_handling                test                test   
feature_reduction               test                test   
hyperparameters                 test                test   
metrics                         test                test   
probability_calibration         test                test   
session_info                    test                test   
target                          test                test   
train                           test                test   
weighting_stratergy             test                test   

                                                         design_state_data  
algorithms               {'RandomForestClassifier': {'model_name': 'Ran...  
feature_generation       {'linear_interactions': [['petal_length', 'sep...  
feature_handling         {'sepal_length': {'feature_name': 'sepal_lengt...  
feature_reduction        {'feature_reduction_method': 'Tree-based', 'nu...  
hyperparameters          {'stratergy': 'Grid Search', 'shuffle_grid': T...  
metrics                  {'optomize_model_hyperparameters_for': 'AUC', ...  
probability_calibration  {'probability_calibration_method': 'Sigmoid - ...  
session_info             {'project_id': '1', 'experiment_id': 'kkkk-11'...  
target                   {'prediction_type': 'Regression', 'target': 'p...  
train                    {'policy': 'Split the dataset', 'time_variable...  
weighting_stratergy      {'weighting_stratergy_method': 'Sample weights...

In [42]:
for key in json_data.keys():
    if json_data[key] is None:
        print(f'Missing value for key: {key}')
        
for element in json_data:
    if element is None:
        print('Missing value in list')

In [43]:
# Extract the target and type of regression to be run
design_state_data = json_data['design_state_data']

In [44]:
target=design_state_data['target']
target

{'prediction_type': 'Regression',
 'target': 'petal_width',
 'type': 'regression',
 'partitioning': True}

In [45]:
prediction_type = target['prediction_type']
target_variable = target['target']
regression_type = target['type']
partitioning_flag = target['partitioning']

In [46]:
# Load the CSV file and extract the features
df = pd.read_csv('iris.csv')
features = df.columns.tolist()
features

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

In [47]:
df.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [50]:
# Initialize a SimpleImputer object with the mean strategy
imputer = SimpleImputer(strategy='most_frequent')

# Fit the imputer to the data
imputer.fit(df[features])

# Apply the missing imputation strategy to the features
ct = ColumnTransformer(
    transformers=[('imputer', SimpleImputer(strategy='most_frequent'), features)]
)
df_imputed = ct.fit_transform(df)
df_imputed

array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
       [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
       [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
       [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
       [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
       [5.4, 3.7, 1.5, 0.2, 'Iris-setosa'],
       [4.8, 3.4, 1.6, 0.2, 'Iris-setosa'],
       [4.8, 3.0, 1.4, 0.1, 'Iris-setosa'],
       [4.3, 3.0, 1.1, 0.1, 'Iris-setosa'],
       [5.8, 4.0, 1.2, 0.2, 'Iris-setosa'],
       [5.7, 4.4, 1.5, 0.4, 'Iris-setosa'],
       [5.4, 3.9, 1.3, 0.4, 'Iris-setosa'],
       [5.1, 3.5, 1.4, 0.3, 'Iris-setosa'],
       [5.7, 3.8, 1.7, 0.3, 'Iris-setosa'],
       [5.1, 3.8, 1.5, 0.3, 'Iris-setosa'],
       [5.4, 3.4, 1.7, 0.2, 'Iris-setosa'],
       [5.1, 3.7, 1.5, 0.4, 'Iris-setosa'],
       [4.6, 3.6, 1.0, 0.2, 'Iri

In [51]:


# Compute feature reduction based on input
if design_state_data['feature_reduction'] == 'corr_with_target':
    corr_with_target = df_imputed.corrwith(df[target])
    features = corr_with_target[abs(corr_with_target) > json_data['corr_threshold']].index.tolist()
elif design_state_data['feature_reduction'] == 'tree_based':
    rf = RandomForestRegressor(n_estimators=100)
    rf.fit(df_imputed.drop(target, axis=1), df_imputed[target])
    feature_importances = pd.Series(rf.feature_importances_, index=features)
    features = feature_importances[feature_importances > json_data['tree_threshold']].index.tolist()
elif design_state_data['feature_reduction'] == 'pca':
    pca = PCA(n_components=json_data['pca_components'])
    pca.fit(df_imputed.drop(target, axis=1))
    df_reduced = pca.transform(df_imputed.drop(target, axis=1))
    features = ['PC{}'.format(i) for i in range(1, json_data['pca_components']+1)]
else:
    features = df.columns.tolist()
    
print("Selected features:", features)

Selected features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']


In [52]:
# Parse the Json and make the model objects
if regression_type == 'linear':
    models = [
        ('lr', LinearRegression())
    ]
    model_params = {
        'lr__fit_intercept': [True, False]
    }
elif regression_type == 'ridge':
    models = [
        ('ridge', Ridge())
    ]
    model_params = {
        'ridge__alpha': np.logspace(-4, 4, 9)
    }
else:
    models = [
        ('rf', RandomForestRegressor())
    ]
    model_params = {
        'rf__n_estimators': [100, 500],
        'rf__max_depth': [None, 5, 10]
    }

print("Models:", models)
print("Model parameters:", model_params)

Models: [('rf', RandomForestRegressor())]
Model parameters: {'rf__n_estimators': [100, 500], 'rf__max_depth': [None, 5, 10]}


In [53]:
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [60]:
df1=df1.transpose()
df1

session_name session_description  \
algorithms                      test                test   
feature_generation              test                test   
feature_handling                test                test   
feature_reduction               test                test   
hyperparameters                 test                test   
metrics                         test                test   
probability_calibration         test                test   
session_info                    test                test   
target                          test                test   
train                           test                test   
weighting_stratergy             test                test   

                                                         design_state_data  
algorithms               {'RandomForestClassifier': {'model_name': 'Ran...  
feature_generation       {'linear_interactions': [['petal_length', 'sep...  
feature_handling         {'sepal_length': {'feature_name': 'sepal_lengt...  
feature_reduction        {'feature_reduction_method': 'Tree-based', 'nu...  
hyperparameters          {'stratergy': 'Grid Search', 'shuffle_grid': T...  
metrics                  {'optomize_model_hyperparameters_for': 'AUC', ...  
probability_calibration  {'probability_calibration_method': 'Sigmoid - ...  
session_info             {'project_id': '1', 'experiment_id': 'kkkk-11'...  
target                   {'prediction_type': 'Regression', 'target': 'p...  
train                    {'policy': 'Split the dataset', 'time_variable...  
weighting_stratergy      {'weighting_stratergy_method': 'Sample weights...

In [58]:
from sklearn.model_selection import train_test_split

X = df[features]
y = df1[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Run the fit and predict on each model using GridSearchCV
for model_name, model in models:
    pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('selector', SelectKBest(mutual_info_regression)),
        ('model', model)
    ])
    params = {
        'selector__k': ['all', 5, 10],
        **model_params
    }
    grid = GridSearchCV(pipe, params, cv=5, n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    score = r2_score(y_test, y_pred)
    
    print(f"Model: {model_name}")
    print(f"Best Parameters: {grid.best_params_}")
    print(f"R-squared Score: {score}\n")


C:\Users\prath\AppData\Local\Temp\ipykernel_34756\2156381699.py:4: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  y = df1[target]


KeyError: "None of [Index(['prediction_type', 'target', 'type', 'partitioning'], dtype='object')] are in the [columns]"